In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
from zoneinfo import ZoneInfo

## Code

In [2]:
jackson = "79bdb7dc-4dec-4e0b-b699-6045af907fd1"
purva = "0221e9b4-4148-4826-a7e5-691c4b30704e"
hamzah = ""

In [3]:
def get_realtime_data(stop_code, api_key):
    url = "https://api.511.org/transit/StopMonitoring?api_key=" + api_key + "&agency=AC&stopCode=" + stop_code
    parsed_url_data = json.loads(requests.get(url).content)
    return parsed_url_data["ServiceDelivery"]["StopMonitoringDelivery"]["MonitoredStopVisit"]

In [8]:
def calculate_delays(bus_line, api_key=purva):
    
    df = pd.read_csv("Line " + bus_line + ".csv")
    delay_dict = {} 
    
    for stop_code in df["stop_code"]:
        bus_list = get_realtime_data(str(stop_code), api_key)
        total_delay = 0
        num_buses = 0
               
        for i in range(len(bus_list)):
            bus_data = bus_list[i]["MonitoredVehicleJourney"]
            
            if (bus_data["LineRef"] == bus_line):
                
                aimed_arrival_time = bus_data["MonitoredCall"]["AimedArrivalTime"]
                expected_arrival_time = bus_data["MonitoredCall"]["ExpectedArrivalTime"]
                
                if (aimed_arrival_time == None or expected_arrival_time == None):
                    continue
                
                aimed = datetime.strptime(aimed_arrival_time, "%Y-%m-%dT%XZ")
                expected = datetime.strptime(expected_arrival_time, "%Y-%m-%dT%XZ")
                delay = (expected - aimed)
            
                if delay.total_seconds() > 0:
                    total_delay += delay.total_seconds()
                    num_buses += 1
                    
        if num_buses > 0:            
            delay_dict[str(stop_code)] = total_delay / num_buses
        
    now = datetime.now().astimezone(ZoneInfo('US/Pacific')).strftime("%m/%d/%Y %H:%M:%S")    
    delays_df = pd.DataFrame(delay_dict.items(), columns=["stop_code", now]) 
    delays_df["stop_code"] = delays_df["stop_code"].astype(int)
    df = df.merge(delays_df, how="left", on="stop_code")
    df.to_csv("Line " + bus_line + ".csv")
    return delay_dict

### Calls

In [9]:
delays_19 = calculate_delays("19", purva)

In [10]:
delays_20 = calculate_delays("20", jackson)

In [11]:
delays_51a = calculate_delays("51A", purva)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)